In [150]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
import cv2, numpy as np
import keras

In [151]:
#загрузили данные
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [152]:
# Размер изображений
img_width, img_height = 32, 32
# Размер мини-выборки
batch_size = 100
# Кол-во изображений для обучения
nb_train_samples = 50000
# Кол-во изображений для теста
nb_test_samples = 10000

In [153]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# one hot encode outputs
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [154]:
# Создание экземпляра модели сети VGG16
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточную часть сети обучать не надо
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
  if layer.name == 'block1_conv1':
    trainable = True
  layer.trainable = trainable

vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [155]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(vgg16_net)
# Преобразуем двумерный массив vgg16 в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_17 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                2570      
Total params: 14,848,586
Trainable params: 14,848,586
Non-trainable params: 0
_________________________________________________________________


In [156]:
# Компилируем составную сеть   'SGD'
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)# Инициализировать оптимизатор RMSprop
epochs = 20
lrate = 0.001
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) #Adam(lr=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, 
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [157]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = epochs, batch_size=batch_size)

Epoch 1/20
500/500 [==============================] - 17s 32ms/step - loss: 1.5561 - accuracy: 0.4458 - val_loss: 0.7625 - val_accuracy: 0.7380
Epoch 2/20
500/500 [==============================] - 16s 31ms/step - loss: 0.7379 - accuracy: 0.7504 - val_loss: 0.6465 - val_accuracy: 0.7743
Epoch 3/20
500/500 [==============================] - 16s 31ms/step - loss: 0.5811 - accuracy: 0.8060 - val_loss: 0.6570 - val_accuracy: 0.7715
Epoch 4/20
500/500 [==============================] - 15s 30ms/step - loss: 0.4934 - accuracy: 0.8351 - val_loss: 0.5178 - val_accuracy: 0.8221
Epoch 5/20
500/500 [==============================] - 16s 31ms/step - loss: 0.4119 - accuracy: 0.8620 - val_loss: 0.4948 - val_accuracy: 0.8340
Epoch 6/20
500/500 [==============================] - 16s 31ms/step - loss: 0.3511 - accuracy: 0.8809 - val_loss: 0.4897 - val_accuracy: 0.8383
Epoch 7/20
500/500 [==============================] - 15s 30ms/step - loss: 0.2960 - accuracy: 0.8999 - val_loss: 0.4969 - val_accuracy:

In [158]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.10%
